In [ ]:
import torch
from torch.utils.data import DataLoader

from src.utils_data import load_PeMS04_flow_data, preprocess_PeMS_data, createLoaders, TimeSeriesDataset
from src.utils_graph import compute_laplacian_with_self_loop
from src.models import TGCN, GRUModel, LSTMModel, train_model

import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Preprocessing data

In [ ]:
df_PeMS, df_distance  = load_PeMS04_flow_data()
n_neighbors = 15
normalization = "divide_by_max"
df_PeMS, adjacency_matrix_PeMS, maximum = preprocess_PeMS_data(df_PeMS, df_distance, init_node=0, n_neighbors=n_neighbors, normalize=True)

In [ ]:
# Define the sliding window size and stride
_window_size = 7
_stride = 1

## TGCN

In [ ]:
train_loader_TGCN, val_loader_TGCN, test_loader_TGCN = createLoaders(df_PeMS, window_size=_window_size, stride=_stride)
model_TGCN = TGCN(adjacency_matrix_PeMS, hidden_dim=32, output_size=len(df_PeMS.columns))

In [ ]:
import matplotlib.pyplot as plt

num_epochs_TGCN = 100
model_path = f"./{normalization}/epoch_{num_epochs_TGCN}/nb_captor_{n_neighbors+1}/TGCN_model.pkl"
_ , valid_losses = train_model(model_TGCN, train_loader_TGCN, val_loader_TGCN, model_path=model_path, num_epochs=num_epochs_TGCN, remove=False)
plt.plot(valid_losses, label="valid_losses")
plt.legend
plt.show()

## Univariate 

### LSTM

In [ ]:
models_univaritate_LSTM = {}

for i in range(n_neighbors+1):
    print(f"LSTMModel {i}")
    train_loader, val_loader, test_loader = createLoaders(pd.DataFrame(df_PeMS.iloc[:, i]), window_size=_window_size, stride=_stride)
    models_univaritate_LSTM[f"LSTMModel {i}"] = { 
                                                "model": LSTMModel(1, 32, 1),
                                                "train_loader": train_loader,
                                                "val_loader": val_loader,
                                                "test_loader": test_loader
    }

In [ ]:
num_epochs_LSTM = 100

for i in range(n_neighbors+1):
    train_model(models_univaritate_LSTM[f"LSTMModel {i}"]["model"], 
                models_univaritate_LSTM[f"LSTMModel {i}"]["train_loader"], 
                models_univaritate_LSTM[f"LSTMModel {i}"]["val_loader"], 
                f"./{normalization}/epoch_{num_epochs_LSTM}/nb_captor_{n_neighbors+1}/univariate_LSTM_model_{i}.pkl", num_epochs=num_epochs_LSTM, remove=False)

### GRU

In [ ]:
models_univaritate_GRU = {}

for i in range(n_neighbors+1):
    print(f"GRUModel {i}")
    train_loader, val_loader, test_loader = createLoaders(pd.DataFrame(df_PeMS.iloc[:, i]), window_size=_window_size, stride=_stride)
    models_univaritate_GRU[f"GRUModel {i}"] = { 
                                                "model": GRUModel(1, 32, 1),
                                                "train_loader": train_loader,
                                                "val_loader": val_loader,
                                                "test_loader": test_loader
    }

In [ ]:
num_epochs_GRU = 100

for i in range(n_neighbors+1):
    train_model(models_univaritate_GRU[f"GRUModel {i}"]["model"], 
                models_univaritate_GRU[f"GRUModel {i}"]["train_loader"], 
                models_univaritate_GRU[f"GRUModel {i}"]["val_loader"], 
                f"./{normalization}/epoch_{num_epochs_GRU}/nb_captor_{n_neighbors+1}/univariate_GRU_model_{i}.pkl", num_epochs=num_epochs_GRU, remove=False)

## Results Multivariate vs Univariate (TGCN VS LSTM - GRU)

In [ ]:
def result_prediction(predictions, actuals):
    from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
    import numpy as np
    
    indices_by_month = []
    EPSILON = 1e-5
    # Créer une liste vide pour stocker les données du tableau
    data = []
    y_pred = predictions[:]
    y_true = actuals[:]

    signe = "-" if np.mean(y_pred - y_true) < 0 else "+"
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = mean_absolute_percentage_error(y_true, y_pred)*100
    if (mape > 1 or mape < 0):
        mape = "ERROR"
    smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))*100
    maape =  np.mean(np.arctan(np.abs((y_true - y_pred) / (y_true + EPSILON))))*100
    
    return [signe, mae, rmse, mape, smape, maape]

In [ ]:
def highlight_col(col, min_val):
    if col.name == "SMAPE":
        color = ['green' if val <= min_val else 'red' for val in col]
        return [f'background-color: {c}' for c in color]
    else :
        return ['' for _ in range(len(col))]


In [ ]:
def highlight_rows(row, min_val):
    color = 'green' if row['SMAPE'] == min_val else 'red'
    return [f'background-color: {color}'] * len(row)

In [ ]:
def test_model_divide_by_max(best_model, test_loader, maximum):
    import numpy as np
    
    # Load the best model and evaluate on the test set
    best_model.double()
    best_model.eval()
    criterion = torch.nn.MSELoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_model.to(device)

    # Evaluate the model on the test set
    test_loss = 0.0
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = (inputs).to(device)
            targets = (targets).squeeze().to(device)
            outputs = best_model(inputs).squeeze()
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            # Save the predictions and actual values for plotting later
            predictions.append(outputs.cpu().numpy())
            actuals.append(targets.cpu().numpy())
    test_loss /= len(test_loader)
    print(f"Test Loss: {test_loss:.4f}")
    # Concatenate the predictions and actuals
    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)

    return (predictions*maximum, actuals*maximum)

In [ ]:
######################################################################
# Center_reduce
######################################################################
def test_model_center_reduce(best_model, test_loader, meanstd_dict):
    import numpy as np
    
    # Load the best model and evaluate on the test set
    best_model.double()
    best_model.eval()
    criterion = torch.nn.MSELoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    best_model.to(device)

    # Evaluate the model on the test set
    test_loss = 0.0
    predictions = []
    actuals = []
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs = ((inputs*meanstd_dict["std"])+meanstd_dict["mean"]).to(device)
            targets = ((targets*meanstd_dict["std"])+meanstd_dict["mean"]).squeeze().to(device)
            outputs = best_model(inputs).squeeze()
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            # Save the predictions and actual values for plotting later
            predictions.append(outputs.cpu().numpy())
            actuals.append(targets.cpu().numpy())
    test_loss /= len(test_loader)
    print(f"Test Loss: {test_loss:.4f}")
    # Concatenate the predictions and actuals
    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)

    return (predictions, actuals)

In [ ]:
import matplotlib.pyplot as plt

######################################################################
# TGCN
######################################################################
# load best model
model_TGCN.load_state_dict(torch.load(f"./{normalization}/epoch_{num_epochs_TGCN}/nb_captor_{n_neighbors+1}/TGCN_model.pkl".format(input)))

# Make predictions
predictions_TGCN, actuals_TGCN = test_model_divide_by_max(model_TGCN, test_loader_TGCN, maximum)


for i in range(n_neighbors+1):
    
    # Save result for each captor
    temp = result_prediction(predictions_TGCN[:, i], actuals_TGCN[:, i])
    temp.append(f"TGCN Model captor {i}")
    data = [temp]
    
######################################################################
# LSTM
######################################################################
    # load best model
    models_univaritate_LSTM[f"LSTMModel {i}"]["model"].load_state_dict(torch.load(f"./{normalization}/epoch_{num_epochs_LSTM}/nb_captor_{n_neighbors+1}/univariate_LSTM_model_{i}.pkl".format(input)))
    
    # Make predictions
    predictions_LSTM, actuals_LSTM = test_model_divide_by_max(models_univaritate_LSTM[f"LSTMModel {i}"]["model"], 
                                    models_univaritate_LSTM[f"LSTMModel {i}"]["test_loader"], 
                                    maximum)
    
    # Save result
    temp = result_prediction(predictions_LSTM, actuals_LSTM)

    temp.append(f"LSTM Model captor {i}")
    data.append(temp)

######################################################################
# GRU
######################################################################
    # Load best model
    models_univaritate_GRU[f"GRUModel {i}"]["model"].load_state_dict(torch.load(f"./{normalization}/epoch_{num_epochs_GRU}/nb_captor_{n_neighbors+1}/univariate_GRU_model_{i}.pkl".format(input)))
    predictions_GRU, actuals_GRU = test_model_divide_by_max(models_univaritate_GRU[f"GRUModel {i}"]["model"], 
                                    models_univaritate_GRU[f"GRUModel {i}"]["test_loader"],
                                    maximum)
    
    # Make predictions
    temp = result_prediction(predictions_GRU, actuals_GRU)

    # Save results
    temp.append(f"GRU Model captor {i}")
    data.append(temp)

######################################################################
# Plots
######################################################################

    plt.figure(figsize=(25, 9))
    plt.plot(predictions_TGCN[:, i], color="red", label="TGCN pred")
    plt.plot(predictions_LSTM, color="black", label="LSTM pred")
    plt.plot(predictions_GRU, color="purple", label="GRU pred" )
    plt.plot(actuals_LSTM, color="orange", label="LSTM true")
    plt.legend(fontsize=15)
    plt.show()

######################################################################
# Dataframe
######################################################################

    df = pd.DataFrame(data, columns=['Signe error', 'MAE', 'RMSE', 'MAPE', 'SMAPE', 'MAAPE', "Captor"])
    
    # Définir le mois comme index du DataFrame
    df.set_index('Captor', inplace=True)

    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', None)

    min_val = df['SMAPE'].min()
    #df = df.style.apply(highlight_col, axis=0, min_val=min_val)
    df = df.style.apply(highlight_rows, axis=1, min_val=min_val)
    display(df)

